In [1]:
from os import walk
import json
import re
from tqdm import tqdm
from utils import pmi, pmi_to_dict_adj, docs_distribution, new_distribution, OptimizedIncrementalPMI
from scoring import compute_scores
import time
import pandas as pd
from textCleaning import get_file_names, extract_year_ipc
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import utils
from utils import docs_distribution, new_distribution, combine_columns, OptimizedIncrementalPMI
import copy
from novelty import ClusterKS

import importlib
importlib.reload(utils)
from utils import docs_distribution, new_distribution, combine_columns, OptimizedIncrementalPMI
import scoring
importlib.reload(scoring)
from scoring import compute_scores
import novelty
importlib.reload(novelty)

c:\Users\edgar\OneDrive\Bureau\Ecole\HEC\A24\BrevetNLP\.conda\Lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.8.0) was trained with spaCy v3.8.0 and may not be 100% compatible with the current version (3.7.5). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\edgar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<module 'novelty' from 'c:\\Users\\edgar\\OneDrive\\Bureau\\Ecole\\HEC\\A24\\BrevetNLP\\PatentNovelty\\novelty\\novelty.py'>

In [ ]:
columns = ['summary', 'abstract', 'background']
w_size = 3
path = "/home/edgarlanoue/data/csvCleaned/" #"C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/cleaned/"
pathOutput = "/home/edgarlanoue/metrics/" #"C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/metrics/"
useClusters=True
yearList = []
ipcList = []

def measureNov(path, pathOutput, columns, w_size, useClusters, yearList, ipcList):
    
    start_time = time.time()  # Record the starting time
    if yearList == [] or ipcList == []:
        tE_names = get_file_names(path+"tE")
        KS_names = get_file_names(path+"KS")
        ES_names = get_file_names(path+"ES")
    
        tE_set = set([extract_year_ipc(string) for string in tE_names])
        KS_set = set([extract_year_ipc(string) for string in KS_names])
        ES_set = set([extract_year_ipc(string) for string in ES_names])
    
        assert(tE_set==KS_set==ES_set)
    
        yearList = sorted(list(set([year for year, ipc in list(tE_set)])))
        ipcList = list(set([ipc for year, ipc in list(tE_set)]))
        print(yearList)
        print(ipcList)
    
    for year in yearList:
        for ipc in ipcList:
            print(f"{year}")
            print(f"    {ipc}")
            tE = pd.read_csv(path+f"tE/{year}_{ipc}_tE_cleaned.csv")
            KS = pd.read_csv(path+f"KS/{year}_{ipc}_KS_cleaned.csv")
            # ES = pd.read_csv(path+f"ES/{year}_{ipc}_ES_cleaned.csv")
    
            # patent = tE[k]
            # file_path = my_path + recette
            # with open(file_path) as json_file:
            #     recipe_dict = json.load(json_file)
    
            #### COLLECTING ALL NECESSARY INFO
    
            # KB_recettes, _ = data_analysis(recipe_dict)
            # print('Knowledge base size : ', len(KB_recettes))
            # if len(KB_recettes) <= 0:
            #     continue
    
            ### Transforming KB into distribution
            application_number = tE["application_number"]
            label = tE["label"]
    
            tE=combine_columns(tE, columns)
            KS=combine_columns(KS, columns)
            # ES=combine_columns(ES, columns)
    
            # Création pmi ES
            # ES_PMI = pmi(ES)
            
            # base_texts = [word for text in tqdm(ES) for word in text.split()]
            # instance_ES_pmi = OptimizedIncrementalPMI(window_size = w_size)
            # print("pmi")
            # instance_ES_pmi.update(base_texts)
            # ES_pmi = instance_ES_pmi.compute_pmi()
            
            # dict_ES_pmi = pmi_to_dict_adj(ES_pmi[0])
            
            KS_matrix, KS_dist, KS_Count_matrix = docs_distribution(baseSpace=KS, tE=tE)
            KS_size = list(range(KS_matrix.shape[0]))
            if useClusters==True:
                KSClusterDiff1000 = ClusterKS(list_know_P=KS_matrix, new_Q= None, N=100, nbPtsPerCluster=1000)
                KSClusterDiff1000.clusterKS()
    
            # ES_matrix,  ES_dist, ES_Count_matrix = docs_distribution(baseSpace=ES, tE=tE, columns=columns)
            # ES_size = list(range(ES_matrix.shape[0]))
            
            ## We set to 0 the distance here for each recipe -- difference needs to estimate distance between all points. 
            #This serves as optim to not calculate for each varaitions but only once since it is the same distance for all KB
            # print('train variations for recipe {} done'.format(recette), ' | train variation size : ', len(train_recettes))
            
            neighborhood_distance  = 0.
            new_ratio_vec = []
            new_bin_vec = []
            uniq_ratio_vec = []
            uniq_bin_vec = []
            diff_ratio_vec = []
            diff_bin_vec = []
            neighborhood_distance_vec = []
            # mean100_vec = []
    
            for i in (range(len(tE))):
                # print("new KS dist (with 1 toEval)")
                select_variation = KS_size + [len(KS_size)+i]
                NewKS_dist, variation_dist = new_distribution(KS_Count_matrix, select_variation)
                
                # print("new ES pmi (with 1 toEval)")
                # # instance_ES_updated_pmi = copy.deepcopy(incremental_pmi_ES_only) # reprise du pmi du ES seulement
                # new_pmi_instance = OptimizedIncrementalPMI(window_size = w_size)
                # update_text = [word for text in tqdm(tE[i]) for word in text.split()] 
                # new_pmi_instance.update(update_text)
                # new_pmi = new_pmi_instance.compute_pmi()
    
    
                # print("compute scores")
                results = compute_scores(KS_matrix, KS_dist, NewKS_dist, variation_dist,# ES_pmi, dict_ES_pmi, new_pmi,
                                            neighbor_dist=neighborhood_distance, useClusters=True, KSCluster=KSClusterDiff1000)
    
                new_ratio = results[0]
                new_bin = results[1]
    
                uniq_ratio = results[2]
                uniq_bin = results[3]
    
                diff_ratio = results[4]
                diff_bin = results[5]
                neighborhood_distance = results[6] #update neighborhood distance so it isn't reset
                # mean100 = results[7]
    
                # surpNew_ratio = results[7]
                # surpNew_bin = results[8]
    
                # surpDiv_ratio = results[9]
                # surpDiv_bin = results[10]
    
                new_ratio_vec.append(new_ratio)
                new_bin_vec.append(new_bin)
    
                uniq_ratio_vec.append(uniq_ratio)
                uniq_bin_vec.append(uniq_bin)
    
                diff_ratio_vec.append(diff_ratio)
                diff_bin_vec.append(diff_bin)
                neighborhood_distance_vec.append(neighborhood_distance)
                # mean100_vec.append(mean100)
    
                if i % 1000 == 0:
                    current_time = time.time()  # Get the current time
                    elapsed_time = current_time - start_time  # Calculate elapsed time
                    if i % 1000 == 0:
                        print(f"{i} on {len(tE)}, time since last print: {elapsed_time:.2f} seconds")
                    start_time = current_time  # Reset the start time for the next interval
                        # Reset the start time to measure time per iteration
                    start_time = time.time()
    
    
            df = pd.DataFrame({
                "application_number": application_number,
                "label": label,
    
                "new_ratio": new_ratio_vec,
                "new_bin": new_bin_vec,
    
                "uniq_ratio": uniq_ratio_vec,
                "uniq_bin": uniq_bin_vec,
    
                "diff_ratio": diff_ratio_vec,
                "diff_bin": diff_bin_vec, 
                "neighboroud_distance": neighborhood_distance_vec,
                # "mean100": mean100_vec
    
                # "surpNew_ratio": surpNew_ratio,
                # "surpNew_bin": surpNew_bin,
    
                # "surpDiv_ratio":surpDiv_ratio,
                # "surpDiv_bin": surpDiv_bin
            })
            df.to_csv(pathOutput + f'/{year}_{ipc}_{columns[0]}_noveltyMetrics.csv', index=False)
            

measureNov(path, pathOutput, columns, w_size, useClusters, yearList, ipcList)

In [4]:
path = "C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/cleaned/"
# path = "/home/edgarlanoue/data/csvCleaned/"

In [5]:
tE_names = get_file_names(path+"tE")
KS_names = get_file_names(path+"KS")
ES_names = get_file_names(path+"ES")

tE_set = set([extract_year_ipc(string) for string in tE_names])
KS_set = set([extract_year_ipc(string) for string in KS_names])
ES_set = set([extract_year_ipc(string) for string in ES_names])

assert(tE_set==KS_set==ES_set)

yearList = [year for year, ipc in list(tE_set)]
ipcList = [ipc for year, ipc in list(tE_set)]

yearList = ["2016"]
ipcList = ["H01L"]

for year in yearList:
    for ipc in ipcList:
        print(f"{year}")
        print(f"    {ipc}")
        tE = pd.read_csv(path+f"tE/{year}_{ipc}_tE_cleaned.csv")
        KS = pd.read_csv(path+f"KS/{year}_{ipc}_KS_cleaned.csv")
        ES = pd.read_csv(path+f"ES/{year}_{ipc}_ES_cleaned.csv")

2016
    H01L


In [7]:
import importlib
import utils
importlib.reload(utils)
from utils import docs_distribution, new_distribution, combine_columns, OptimizedIncrementalPMI

In [8]:
application_number = tE["application_number"]
label = tE["label"]

In [9]:
tE=combine_columns(tE, ["claims"])
KS=combine_columns(KS, ["claims"])
ES=combine_columns(ES, ["claims"])

In [10]:
import importlib
import utils
importlib.reload(utils)
from utils import docs_distribution, new_distribution, combine_columns, OptimizedIncrementalPMI, pmi
from tqdm import tqdm

In [18]:
# base_texts = ' '.join(ES[:1000]).split()
base_texts = [word for text in tqdm(ES) for word in text.split()]
# ES_pmi = pmi(base_texts, w_size=3)
# ES_pmi

100%|██████████| 284544/284544 [00:38<00:00, 7424.28it/s] 


Pmi le mien vs défaut

In [19]:
instOptPMI = OptimizedIncrementalPMI(3)
instOptPMI.update(base_texts)
dict_optPMI = instOptPMI.compute_pmi()

100%|██████████| 11075532/11075532 [02:20<00:00, 78686.81it/s] 


In [13]:
pmi_default = pmi(base_texts, 3)

In [14]:
{key: value[0] for key, value in dict_optPMI[0].items()} == dict(pmi_default)

False

In [21]:
len(dict_optPMI[0])

11075532

In [13]:
print(sorted({key: value[0] for key, value in dict_optPMI[0].items()}) == sorted(dict(pmi_default)))
print(sorted({key: value[0] for key, value in dict_optPMI[0].items()}.values()) == sorted(dict(pmi_default).values()))


True
False


PMI, les deux sont pareils à 1 secondes prêts... le mien est utile puisqu'il segmente les taches, rendant un PMI futur beaucoup plus rapide à calculer, cependant, pu besoin puisqu'on compare simplement l'intersection entre l'ES et le nouveau

In [9]:
# baseTexts_update = [tE[0]]
# update_text = ' '.join(baseTexts_update).split()
# new_pmi = OptimizedIncrementalPMI(window_size=3)
# new_pmi.update(update_text)
# print(new_pmi.bigram_counts)
# print(new_pmi.word_counts)
# print(new_pmi.total_words)

# inutile puisqu'on calcule le PMI du nouveau texte directement

Counter({('dielectric', 'layer'): 45, ('layer', 'layer'): 42, ('layer', 'dielectric'): 40, ('encapsulation', 'layer'): 24, ('light', 'absorb'): 18, ('light', 'layer'): 18, ('absorb', 'layer'): 18, ('layer', 'comprise'): 17, ('layer', 'silicon'): 15, ('silicon', 'layer'): 15, ('diffusion', 'region'): 14, ('layer', 'open'): 12, ('nitride', 'layer'): 12, ('dielectric', 'dielectric'): 12, ('fourth', 'dielectric'): 12, ('fourth', 'layer'): 12, ('layer', 'encapsulation'): 11, ('comprise', 'layer'): 9, ('layer', 'monocrystalline'): 9, ('monocrystalline', 'semiconductor'): 9, ('monocrystalline', 'layer'): 9, ('semiconductor', 'layer'): 9, ('open', 'extend'): 9, ('extend', 'vertically'): 9, ('layer', 'light'): 9, ('comprise', 'silicon'): 9, ('additional', 'open'): 9, ('blanket', 'layer'): 9, ('layer', 'fourth'): 9, ('photodetector', 'comprise'): 8, ('comprise', 'dielectric'): 8, ('layer', 'photodetector'): 7, ('dielectric', 'monocrystalline'): 6, ('layer', 'semiconductor'): 6, ('open', 'vertica

In [22]:
baseTexts_update = [tE[0]]
update_text = [word for text in tqdm(baseTexts_update) for word in text.split()] #' '.join(baseTexts_update).split()

new_pmi = OptimizedIncrementalPMI(window_size=3)
new_pmi.update(update_text)
newpmi_PMI = new_pmi.compute_pmi()

100%|██████████| 260/260 [00:00<00:00, 176288.24it/s]


In [26]:
# base_texts_set = set(base_texts)
# update_text_set = set(update_text)

base_bigram_set = set(instOptPMI.bigram_counts.keys())

update_bigram_set = set(new_pmi.bigram_counts.keys())

common_bigram_set = update_bigram_set & base_bigram_set
# common_word_set = update_text_set & base_texts_set


import math

# 20aine de secondes pour trouver ce qu'ils ont en commun

In [24]:
print(len(common_bigram_set))
# print(len(common_word_set))

242


In [43]:
filtered_pmi_values = [
    (bigram, value) for bigram, value in pmi_default if bigram in common_bigram_set
]

In [44]:
filtered_pmi_dict = {
    key: value for key, value in dict_optPMI[0].items() if key in common_bigram_set
}

In [45]:
filtered_pmi_dict.get(("least", "one"))

[4.443215690345573, 5278, 7836, 3844]

In [46]:
dict_optPMI[3]

468114

In [47]:
import math
np.log2(3844/(7836*5278)*468114*1/(3 - 1))

4.443215690345575

In [48]:
dict_optPMI[0].get(("least", "one"))

[4.443215690345573, 5278, 7836, 3844]

In [49]:
next((value for (key, value) in pmi_default if key == ("least", "one")), None)

4.443215690345575

In [50]:
dict(pmi_default).get(("least", "one"))

4.443215690345575

In [51]:
dict(filtered_pmi_values).get(("least", "one"))

4.443215690345575

In [90]:
print(len(sorted(dict(filtered_pmi_values).values())))
print(len(sorted({key: value[0] for key, value in filtered_pmi_dict.items()}.values())))

106
106


In [128]:
print(sorted(dict(filtered_pmi_values).values())[:10])
print(sorted({key: value for key, value in filtered_pmi_dict.items()}.values())[:10])

[-2.699200249602903, -1.8124416965666938, -1.754979957130459, -1.5948829513290939, -1.3919907011014843, -1.3458672918087196, -1.2157757753438077, -1.070062606214119, -1.0587287346773877, -0.9805644553750206]
[[-2.699200249602903, 576, 5278, 2], [-1.8124416965666938, 3079, 267, 1], [-1.754979957130459, 243, 6502, 2], [-1.5948829513290939, 993, 712, 1], [-1.3919907011014843, 3079, 798, 4], [-1.3458672918087196, 6502, 183, 2], [-1.2157757753438077, 103, 5278, 1], [-1.0700626062141225, 798, 3079, 5], [-1.0587287346773877, 491, 993, 1], [-0.9805644553750206, 3079, 150, 1]]


In [85]:
sorted(dict(filtered_pmi_values).values())==sorted({key: value[0] for key, value in filtered_pmi_dict.items()}.values())

False

In [52]:
filtered_pmi_dict

{('least', 'one'): [4.443215690345573, 5278, 7836, 3844],
 ('comprise', 'least'): [1.0661517565192327, 6502, 5278, 307],
 ('comprise', 'layer'): [-0.30988914879132423, 6502, 3079, 69],
 ('layer', 'layer'): [3.069399019213746, 3079, 3079, 340],
 ('region', 'region'): [3.712286739899108, 1500, 1500, 126],
 ('comprise', 'one'): [0.6727227167859979, 6502, 7836, 347],
 ('region', 'comprise'): [-0.680474520766829, 1500, 6502, 26],
 ('surface', 'surface'): [2.7919543772150064, 2534, 2534, 190],
 ('layer', 'comprise'): [0.47640415773845035, 3079, 6502, 119],
 ('oxide', 'layer'): [2.324489820947946, 258, 3079, 17],
 ('comprise', 'diode'): [-0.07334143699763018, 6502, 303, 8],
 ('diode', 'pair'): [0.9068258765095081, 303, 412, 1],
 ('bottom', 'surface'): [4.268772027801649, 345, 2534, 72],
 ('layer', 'open'): [0.1160299428637721, 3079, 491, 7],
 ('top', 'surface'): [4.663685257994869, 379, 2534, 104],
 ('bottom', 'top'): [4.424956298050212, 345, 379, 12],
 ('surface', 'top'): [2.663685257994869,

In [30]:
pmi_base = dict_optPMI[0].copy()

dif_words_bigrams = {word for tup in dif_bigram_set for word in tup}
filtered_word_counts = {key: value for key, value in instOptPMI.word_counts.items() if key in dif_words_bigrams}
pmi_newPMI = new_pmi.compute_pmi()[0]

# inter_bigrams = {key: value for key, value in pmi_base.items() if key in new_pmi.bigram_counts.keys()}
# inter_words = {key: value for key, value in instOptPMI.word_counts.items() if key in update_text_set}
print("here")
new_bigrams = set(new_pmi.bigram_counts.keys())
filter_pmi = {key: value for key, value in pmi_base.items() if key in new_bigrams}
# Add missing keys from pmi_newPMI to filter_pmi with value [0, 0, 0, 0]
filter_pmi.update({key: [0, filtered_word_counts.get(key[0], 0), filtered_word_counts.get(key[1], 0), 0] for key in pmi_newPMI if key not in filter_pmi})
# dif_newPMI = {key: value for key, value in pmi_newPMI.items() if key in dif_bigram_set}
print("here2")
modified_new_values = {}
# modified_new_values.update(dif_newPMI)
# print(modified_new_values)
# Use dictionary comprehension to sum corresponding elements of the lists in both dictionaries
modified_new_values = {key: [a + b for a, b in zip(pmi_newPMI[key], filter_pmi[key])]
                 for key in pmi_newPMI}

print("here3")
# Update the first element of each list with the given formula
for key, value in modified_new_values.items():
    value[0] = math.log2(value[3] / (value[1] * value[2])) + math.log2((new_pmi.total_words + instOptPMI.total_words) * 0.5)
modified_new_values


print("here4")
for key, value in pmi_base.items():
    value[0] = value[0] - math.log2(instOptPMI.total_words) + math.log2(instOptPMI.total_words + new_pmi.total_words)

pmi_base.update(modified_new_values)

# vingatine de secondes

100%|██████████| 260/260 [00:00<00:00, 131976.16it/s]


here
here2
here3
here4


In [58]:
import importlib
import surprise
import divergences
importlib.reload(surprise)
importlib.reload(utils)
importlib.reload(divergences)
from surprise import Surprise
from utils import pmi_to_dict_adj, pmi_to_dict_adj_dict

In [ ]:
instanceSurp = Surprise(modified_new_values)

In [ ]:
instanceSurp.uniq_surprise_adj(pmi_to_dict_adj(filter_pmi), eps=0)

In [ ]:
dict_known = pmi_to_dict_adj_dict(dict_optPMI[0])

New

In [55]:
new_pmi

In [59]:
instSurp = Surprise(newpmi_PMI)

In [61]:
dict_known = pmi_to_dict_adj_dict(dict_optPMI[0])
dict_new = pmi_to_dict_adj_dict(newpmi_PMI[0])

In [63]:
gCV = instSurp.get_common_vectors_adj(dict_known, dict_new, epsilon = 0)

100%|██████████| 8042/8042 [00:15<00:00, 510.99it/s]
